In [ ]:
# train yolov7
!ls

In [ ]:
%cd yolov7

In [ ]:
!ls

In [ ]:
!python train.py --freeze 29 --batch-size 32 --data "../data/FineTuned_YOLO/data.yaml" --img 1080 1920 --cfg cfg/training/yolov7-e6e.yaml --weight "./yolov7-e6e.pt" --name yolov7-custom-freeze --epoch 200
!python detect.py --weights "./runs/train/yolov7-custom-freeze/weights/best.pt" --conf 0.25 --source ./video1_test.mp4

In [ ]:
https://learnopencv.com/fine-tuning-yolov7-on-custom-dataset/